In [1]:
import torch
import random

import pandas as pd

from torch.utils.data import Dataset
import torch.nn.functional as F
import torch.nn as nn

from torch.optim import AdamW

import torch_geometric.transforms as T

from torch_geometric.data import Batch

from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.nn import global_add_pool
from torch_geometric.nn import GraphConv
from torch.utils.data import DataLoader

from pathlib import Path

from tqdm import tqdm

In [7]:
import sys
import os
cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
sys.path.append(parent_dir)
from DataPipeline.dataset import ZincSubgraphDataset, custom_collate
from Model.GNN1 import ModelWithEdgeFeatures
from Model.metrics import pseudo_accuracy_metric, pseudo_recall_for_each_class, pseudo_precision_for_each_class
from Train.training import complete_training

In [8]:
datapath = Path('..') / 'DataPipeline/data/preprocessed_graph_no_I_Br_P.pt'
dataset = ZincSubgraphDataset(data_path = datapath)

Dataset encoded with size 7


In [19]:
dataloader = DataLoader(dataset, batch_size=128, shuffle=True, num_workers=0, collate_fn=custom_collate)

In [20]:
#load checkpoint
encoding_size = 7
device = torch.device('cuda:0')

model = ModelWithEdgeFeatures(num_classes = encoding_size, in_channels=encoding_size, hidden_channels_list=[64, 128, 256, 512, 512], mlp_hidden_channels=512, edge_channels=4, use_dropout=False)
optimizer = AdamW(model.parameters(), lr=0.0001)

checkpoint_path = Path('..') / 'Train' / 'checkpoint_epoch_96_balanced_10-4.pt'
checkpoint = torch.load(checkpoint_path)
checkpoint = {key: tensor.to(device) if isinstance(tensor, torch.Tensor) else tensor
              for key, tensor in checkpoint.items()}
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


In [21]:
criterion = nn.CrossEntropyLoss()
model.to(device)


complete_training(model, optimizer, criterion, dataloader, device=torch.device('cuda:0'), n_epochs=5, name='finetuning_model', encoding_size=encoding_size)

Training:   0%|          | 0/1845 [00:00<?, ?batch/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!